# Plagiarism Detection using Transformers based models   


In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()


In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                                     title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                                     Online Food Dataset                                  3KB  2024-03-02 18:50:30          15245        313  0.9411765        
sukhmandeepsinghbrar/car-price-prediction-dataset                       Car Price Prediction Dataset                       141KB  2024-03-28 08:27:39            727         28  1.0              
prishasawhney/imdb-dataset-top-2000-movies                              IMDB Dataset (Top 2000 movies)                      75KB  2024-03-27 18:51:36            679         24  1.0              
syedanwarafridi/vehicle-s

In [ ]:
!kaggle datasets download -d naeemrehmat/cleaned-cord-19

 99% 523M/528M [00:09<00:00, 64.7MB/s]
100% 528M/528M [00:09<00:00, 57.9MB/s]


In [ ]:
!unzip "cleaned-cord-19.zip"

Archive:  cleaned-cord-19.zip
  inflating: biorxiv_clean.csv       
  inflating: clean_comm_use.csv      
  inflating: clean_noncomm_use.csv   
  inflating: clean_pmc.csv           


checking if the dataset could be handled by the collab machine without memory overflow


In [ ]:
source_data = pd.read_csv("clean_pmc.csv", low_memory=False, nrows=1000)

In [ ]:
source_data

,paper_id,title,authors,affiliations,abstract,text,bibliography,raw_authors,raw_bibliography
0,14572a7a9b3e92b960d92d9755979eb94c448bb5,Immune Parameters of Dry Cows Fed Mannan Oligo...,"S T Franklin, M C Newman, K E Newman, K I Meek","S T Franklin (University of Kentucky, 40546-02...",Abstract\n\nThe objective of this study was to...,INTRODUCTION\n\nThe periparturient period is a...,Immune response of pregnant heifers and cows t...,"[{'first': 'S', 'middle': ['T'], 'last': 'Fran...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Immune ..."
1,bb790e8366da63c4f5e2d64fa7bbd5673b93063c,Discontinuous Transcription or RNA Processing ...,"Beate Schwer, Paolo Vista, Jan C Vos, Hendrik ...","Beate Schwer, Paolo Vista, Jan C Vos, Hendrik ...",NaN,Discontinuous\n\nTranscription or RNA Processi...,Poly (riboadenylic acid) preferentially inhibi...,"[{'first': 'Beate', 'middle': [], 'last': 'Sch...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Poly (r..."
2,24f204ce5a1a4d752dc9ea7525082d225caed8b3,NaN,NaN,NaN,NaN,Letter to the Editor\n\nThe non-contact handhe...,Novel coronavirus is putting the whole world o...,[],"{'BIBREF0': {'ref_id': 'b0', 'title': 'Novel c..."
3,ab78a42c688ac199a2d5669e42ee4c39ff0df2b8,A real-time convective PCR machine in a capill...,"Yi-Fan Hsieh, Da-Sheng Lee, Ping-Hei Chen, Sha...","Yi-Fan Hsieh (National Taiwan University, 106,...","Abstract\n\nThis research reports the design, ...",Introduction\n\nMullis et al. developed the po...,"The Polymerase Chain Reaction, K B Mullis, F F...","[{'first': 'Yi-Fan', 'middle': [], 'last': 'Hs...","{'BIBREF0': {'ref_id': 'b0', 'title': 'The Pol..."
4,31105078a2953217223699d09c6a80d0f5edfdf6,Infecciones virales graves en pacientes inmuno...,"A Díaz, R Zaragoza, R Granada, M Salavert",A Díaz (Hospital Universitario Virgen del Rocí...,Abstract\n\nRecibido el 21 de diciembre de 201...,"\n\nFurthermore, ventilator-associated pneumon...",High prevalence of respiratory viral infecions...,"[{'first': 'A', 'middle': [], 'last': 'Díaz', ...","{'BIBREF0': {'ref_id': 'b0', 'title': 'High pr..."
...,...,...,...,...,...,...,...,...,...
995,25351c9bf4321578cd9a1c98745f8d255ceab3fd,Immunity and age: living in the past?,"David L Woodland, Marcia A Blackman","David L Woodland (Trudeau Institute, 12983, Sa...",NaN,\n\nIncreasing age is associated with a decrea...,Age-related changes in lymphocyte development ...,"[{'first': 'David', 'middle': ['L'], 'last': '...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Age-rel..."
996,f3c76a89e5017f280bd3cd75859acbf4f5c40352,Electrophoretic RNA genomic profiles of Brazil...,"Elisabete Takiuchi, Rubia Macedo, Fernanda Kun...",Elisabete Takiuchi (Federal University of Para...,Abstract\n\nPicobirnavirus (PBV) belongs to th...,"Introduction\n\nPicobirnavirus (PBV), the only...",A new bi-segmented double stranded RNA virus i...,"[{'first': 'Elisabete', 'middle': [], 'last': ...","{'BIBREF0': {'ref_id': 'b0', 'title': 'A new b..."
997,a2baa4e07df066fb5c7cd8cf4445966cfcdfcf30,A novel feline norovirus in diarrheic cats,"Barbara Di Martino, Federica Di Profio, Irene ...",Barbara Di Martino (Università degli Studi di ...,Abstract\n\nBy screening a collection of fecal...,"Introduction\n\nNoroviruses (NoVs), Caliciviri...",Characterization of new recombinant noroviruse...,"[{'first': 'Barbara', 'middle': ['Di'], 'last'...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Charact..."
998,b9bda4ac723da46f0a97cb515f56fcdb1588f162,Comparison of Bovine Coronavirus (BCV) Antigen...,"Khalid A Hussain, Johannes Storz,&apos; And, K...","Khalid A Hussain (Louisiana State University, ...",Abstract\n\nMonoclonal antibodies (MAbs) again...,\n\nBovine coronavirus (BCV) is a member of th...,Proceedings of the International Symposium on ...,"[{'first': 'Khalid', 'middle': ['A'], 'last': ...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Proceed..."


In [ ]:
source_data.columns

Index(['paper_id', 'title', 'authors', 'affiliations', 'abstract', 'text',
       'bibliography', 'raw_authors', 'raw_bibliography'],
      dtype='object')

In [ ]:
def preprocess_data(data_path, sample_size):

  # Read the data from specific path
  data = pd.read_csv(data_path, low_memory=False)

  # Drop articles without Abstract & text
  data = data.dropna(subset = ['abstract']).reset_index(drop = True)
  data = data.dropna(subset = ['text']).reset_index(drop = True)

  # Get "sample_size" random articles
  # data = data.sample(sample_size)[['abstract', 'paper_id']]
  data = data.sample(len(data))[['text', 'paper_id']]


  return data


data_path = "clean_pmc.csv"
source_data = preprocess_data(data_path, 20000)

In [ ]:
output_file_path = "preprocessed_data.csv"
source_data.to_csv(output_file_path , index=False)

In [1]:
import pandas as pd
from tqdm import tqdm

## Load BERT Model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [151]:
import shutil
import os

def copy_file_to_workspace(source_path, destination_path):
    try:
        shutil.copy(source_path, destination_path)
        print(f"File copied successfully from {source_path} to {destination_path}")
    except FileNotFoundError:
        print("File not found.")
    except PermissionError:
        print("Permission denied.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
source_drive_path = "/content/drive/MyDrive/Plagiarism Detector/preprocessed_data.csv"
destination_colab_path = "/content/preprocessed_data.csv"

copy_file_to_workspace(source_drive_path, destination_colab_path)

source_drive_path2 = "/content/drive/MyDrive/Plagiarism Detector/database.csv"
destination_colab_path2 = "/content/database.csv"

copy_file_to_workspace(source_drive_path2, destination_colab_path2)


File copied successfully from /content/drive/MyDrive/Plagiarism Detector/preprocessed_data.csv to /content/preprocessed_data.csv
File copied successfully from /content/drive/MyDrive/Plagiarism Detector/database.csv to /content/database.csv


In [152]:
# !pip uninstall -y transformers
!pip install transformers[torch]
!pip install accelerate -U


!pip install accelerate>=0.21.0


In [ ]:
# !pip install transformers[torch]

In [153]:
import pandas as pd
data_path = "preprocessed_data.csv"
data_path2 = "database.csv"


source_data = pd.read_csv(data_path, low_memory=False, nrows=1000)
vector_database = pd.read_csv(data_path, low_memory=False, nrows=1000)


In [154]:
# !pip -q install transformers[torch]
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer,  AutoModelForSequenceClassification

# Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

In [155]:
import torch
from transformers import BertTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [156]:
model_path = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_path,
                                          do_lower_case=True)

model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                          output_attentions=False,
                                                          output_hidden_states=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data from CSV file
df = source_data

# Assuming 'label' is your target variable and the rest are features
X = df.drop('paper_id', axis=1)
y = df['paper_id']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now, X_train and y_train are your training data

In [ ]:
# from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# import torch

# # Load the data
# data = source_data
# data['label'] = 0

# # Split the data into training and validation sets
# train_texts, val_texts, train_labels, val_labels = train_test_split(data['text'], data['label'], test_size=0.2)

# # Initialize the tokenizer
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# # Tokenize the texts
# train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
# val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

# # Convert the labels to tensors
# label_encoder = LabelEncoder()
# # Fit the LabelEncoder on all possible labels
# label_encoder = LabelEncoder()
# label_encoder.fit(pd.concat([y_train, y_test]))

# # Then transform the labels into integers
# y_train_encoded = label_encoder.transform(y_train)
# y_test_encoded = label_encoder.transform(y_test)
# # Convert the tokenized data to tensors
# train_input_ids = torch.tensor(train_encodings['input_ids'])
# train_attention_mask = torch.tensor(train_encodings['attention_mask'])

# val_input_ids = torch.tensor(val_encodings['input_ids'])
# val_attention_mask = torch.tensor(val_encodings['attention_mask'])

# # Create a TensorDataset from train_encodings and y_train
# train_dataset = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels)
# val_dataset = torch.utils.data.TensorDataset(val_input_ids, val_attention_mask, val_labels)

# train_labels = torch.tensor(label_encoder.fit_transform(train_labels))
# val_labels = torch.tensor(label_encoder.transform(val_labels))

# # Create the datasets
# train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
# val_dataset = torch.utils.data.TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)

# # Define the model
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# # Define the training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=64,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='./logs',
# )

# # Define the trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
# )

# # Train the model
# trainer.train()

# # Evaluate the model
# trainer.evaluate()

In [ ]:
# from torch import Tensor
# from torch.utils.data import TensorDataset, DataLoader
# from transformers import BertTokenizerFast
# from sklearn.preprocessing import LabelEncoder

# # Assuming X_train and y_train are your training data
# # X_train = ...
# # y_train = ...

# # Load the BERT tokenizer
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# # Tokenize the training dataset
# train_encodings = tokenizer(list(X_train['text']), truncation=True, padding=True)
# test_encodings = tokenizer(list(X_test['text']), truncation=True, padding=True)



# # Convert your labels into integers
# label_encoder = LabelEncoder()
# y_train_encoded = label_encoder.fit_transform(y_train)
# y_test_encoded = label_encoder.fit_transform(y_test)

# # Convert your labels into PyTorch tensors
# y_train_tensor = Tensor(y_train_encoded)
# y_test_tensor = Tensor(y_test_encoded)

# # Convert the tokenized data to tensors
# input_ids = Tensor(train_encodings['input_ids'])
# attention_mask = Tensor(train_encodings['attention_mask'])

# input_ids_test = Tensor(test_encodings['input_ids'])
# attention_mask_test = Tensor(test_encodings['attention_mask'])

# # Create a TensorDataset from train_encodings and y_train
# train_dataset = TensorDataset(input_ids, attention_mask, y_train_tensor)
# val_dataset = TensorDataset(input_ids_test, attention_mask_test , y_test_tensor)

# # Create a DataLoader
# train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size= 16, shuffle =True)

In [ ]:
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
#     acc = accuracy_score(labels, preds)
#     return {
#         'accuracy': acc,
#         'f1': f1,
#         'precision': precision,
#         'recall': recall
#     }

In [ ]:
# from transformers import Trainer, TrainingArguments

# # Define the training arguments
# training_args = TrainingArguments(
#     output_dir='./results',          # output directory
#     num_train_epochs=3,              # total number of training epochs
#     per_device_train_batch_size=16,  # batch size per device during training
#     per_device_eval_batch_size=64,   # batch size for evaluation
#     warmup_steps=500,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
# )

# # Define the trainer
# trainer = Trainer(
#     model=model,                         # the instantiated 🤗 Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=train_dataset,         # training dataset
#     eval_dataset=val_dataset,            # evaluation dataset
#     compute_metrics=compute_metrics,     # function that computes metrics
# )

# # Train the model
# trainer.train()

# # Evaluate the model
# trainer.evaluate()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import ConfusionMatrixDisplay

# # Get the predictions on the test dataset
# predictions = trainer.predict(tokenized_test_dataset)

# # Get the true labels and predicted labels
# true_labels = predictions.label_ids
# pred_labels = predictions.predictions.argmax(-1)

# # Compute the metrics
# precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='binary')
# conf_mat = confusion_matrix(true_labels, pred_labels)

# # Print the metrics
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

# # Plot the confusion matrix
# plt.figure(figsize=(10, 7))
# sns.heatmap(conf_mat, annot=True, fmt='d')
# plt.title('Confusion Matrix')
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

In [159]:
def create_vector_from_text(tokenizer, model, text, MAX_LEN = 510):

    input_ids = tokenizer.encode(
                        text,
                        add_special_tokens = True,
                        max_length = MAX_LEN,
                        truncation= True,
                   )

    results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long",
                              truncating="post", padding="post")

    # Remove the outer list.
    input_ids = results[0]

    # Create attention masks
    attention_mask = [int(i>0) for i in input_ids]

    # Convert to tensors.
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)

    # Add an extra dimension for the "batch" (even though there is only one
    # input in this batch.)
    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)

    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()

    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers.
    with torch.no_grad():
        logits, encoded_layers = model(
                                    input_ids = input_ids,
                                    token_type_ids = None,
                                    attention_mask = attention_mask,
                                    return_dict=False)

    layer_i = 12 # The last BERT layer before the classifier.
    batch_i = 0 # Only one input in the batch.
    token_i = 0 # The first token, corresponding to [CLS]

    # Extract the embedding.
    vector = encoded_layers[layer_i][batch_i][token_i]

    # Move to the CPU and convert to numpy ndarray.
    vector = vector.detach().cpu().numpy()

    return(vector)



,

  



In [160]:
import numpy as np

def create_vector_database(data):

    # The list of all the vectors
    vectors = []

    # Get overall text data
    source_data = data.text.values

    # Loop over all the comment and get the embeddings
    for text in tqdm(source_data):

        # Get the embedding
        vector = create_vector_from_text(tokenizer, model, text)

        #add it to the list
        vectors.append(vector)

    data["vectors"] = vectors
    data["vectors"] = data["vectors"].apply(lambda emb: np.array(emb))
    data["vectors"] = data["vectors"].apply(lambda emb: emb.reshape(1, -1))


    return data

In [ ]:
from transformers import Trainer, TrainingArguments



tokenized_train_dataset = train_dataset

# Split the data into train and test sets
train_data, test_data = train_test_split(source_data, test_size=0.2, random_state=42)

# Assuming all data in the test set is non-plagiarized (label = 0)
test_data['label'] = 0
train_data['label'] = 0


# Convert test data to a list of dictionaries
test_dataset = test_data[['text', 'label']].to_dict('records')

# Convert train data to a list of dictionaries
train_dataset = train_data[['text', 'label']].to_dict('records')

# Tokenize the test data
tokenizer = AutoTokenizer.from_pretrained(model_path)

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        label = item['label']
        encoding = self.tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'text': torch.tensor(text)
        }

# Create PyTorch Dataset objects
test_dataset = TextDataset(test_dataset)
train_dataset = TextDataset(train_dataset)

# You can now use the datasets directly
tokenized_test_dataset = test_dataset
tokenized_train_dataset = train_dataset

# Fine-tune the model
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)  # 2 for binary classification

# Create data loaders
train_loader = DataLoader(tokenized_train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(tokenized_test_dataset, batch_size=16)

# ... (rest of the code remains the same)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
tokenized_train_dataset = train_dataset

# Split the data into train and test sets
train_data, test_data = train_test_split(source_data, test_size=0.2, random_state=42)

# Assuming all data in the test set is non-plagiarized (label = 0)
test_data['label'] = 0
train_data['label'] = 0


# Convert test data to a list of dictionaries
test_dataset = test_data[['text', 'label']].to_dict('records')

# Convert train data to a list of dictionaries
train_dataset = train_data[['text', 'label']].to_dict('records')

# Tokenize the test data
tokenizer = AutoTokenizer.from_pretrained(model_path)

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        label = item['label']
        encoding = self.tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label)
        }

# Create PyTorch Dataset objects
test_dataset = TextDataset(test_dataset)
train_dataset = TextDataset(train_dataset)

# You can now use the datasets directly
tokenized_test_dataset = test_dataset
tokenized_train_dataset = train_dataset

# Fine-tune the model
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)  # 2 for binary classification

# Create data loaders
train_loader = DataLoader(tokenized_train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(tokenized_test_dataset, batch_size=16)



# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    conf_mat = confusion_matrix(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'confusion_matrix': str(conf_mat)  # Convert the confusion matrix to a string
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)

# Print the evaluation results
print(f"Evaluation results: {eval_results}")

# Save the fine-tuned model
output_model_path = "finetuned_plagiarism_detection_model"
trainer.save_model(output_model_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Confusion Matrix
1,No log,0.000295,1.000000,0.000000,0.000000,0.000000,[[200]]
2,No log,0.000156,1.000000,0.000000,0.000000,0.000000,[[200]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "[[200]]" of type <class 'str'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted sam

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay

# Get the predictions on the test dataset
predictions = trainer.predict(tokenized_test_dataset)

# Get the true labels and predicted labels
true_labels = predictions.label_ids
pred_labels = predictions.predictions.argmax(-1)

# Compute the metrics
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='binary')
conf_mat = confusion_matrix(true_labels, pred_labels)

# Print the metrics
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_mat, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:

source_data.head(20)

,text,paper_id,label
0,\n\nAcute respiratory tract infections (ARIs) ...,a5dcd055401621a938dd835c1094db2ed86c33da,0
1,"Introduction\n\nRecently, graphical techniques...",d7855f0e37fe34ab8dd7d405ceb64fab485f0258,0
2,Introduction\n\nThe carboxylic acid derivative...,064c04b451db7ee813746dde89ad46963d39a8a3,0
3,"\n\nLinda Humphries F or hundreds of years, su...",d1eb75bf85d281d6d99e6db97573e8e7bc0bf10c,0
4,"\n\nStructured, organized, government-sponsore...",4b34c9376a254a376e289a131d3d9ba039733479,0
5,Introduction\n\nAcute respiratory tract infect...,2480b2d6683049fbd9522d1b2e8dd1ea006fb122,0
6,Introduction\n\nSevere acute respiratory syndr...,861331c3e2e02fc2ad37835a72e81dd2750193df,0
7,Introduction\n\nChina adopted its first nation...,0a9aa6532887ae8de991dacd31b6ba7c9b5b4817,0
8,"INTRODUCTION\n\nPrimary demyelination, i.e., t...",1b2f4e588bda174f17b0e68edb0bcdd4e43728f0,0
9,"Introduction\n\nFrom May to December of 2003, ...",6e2c9f2f4eb33812e4458660db84d6fcb2b92bd2,0


In [162]:
data = pd.read_csv(data_path, low_memory=False, nrows=100)


In [ ]:
vector_database = create_vector_database(source_data)

In [171]:
vector_database = create_vector_database(data)


100%|██████████| 100/100 [03:34<00:00,  2.15s/it]


In [163]:
vector_database = pd.read_csv(data_path2, low_memory=False, nrows=100)

In [174]:
vector_database.sample(5)

,text,paper_id,vectors,similarity
2,Introduction\n\nThe carboxylic acid derivative...,064c04b451db7ee813746dde89ad46963d39a8a3,"[[-0.57619905, -0.01567056, 0.046126388, 0.385...",0.743244
48,Introduction\n\nPorcine epidemic diarrhea (PED...,78d857b12433f5316bb9fabf682e898fb3cf46da,"[[-0.42679304, -0.68075573, -0.87815255, -0.36...",0.806527
76,Introduction\n\nThe influenza pandemic of 1918...,d03cf2f21e08b7e0df78531221da6c06915e1b50,"[[-0.46677878, -0.45238382, -0.5642016, 0.2866...",0.773302
63,Introduction\n\nOne important factor that affe...,9b07951ffeb24912f35e85556e35108481b02755,"[[-0.942179, -0.31781122, -0.5125652, 0.224066...",0.779718
58,Introduction\n\nWhile the presentation of a pa...,4c9148a63a61aadcbfc86d77af99f8c6602e62cb,"[[-0.6614695, -0.36978114, -0.62984526, -0.599...",0.832347


In [ ]:
output_file_path = "database.csv"
vector_database.to_csv(output_file_path, index=False)


# Language detector and translation

In [ ]:
!pip -q install sentencepiece
from transformers import MarianMTModel, MarianTokenizer

**Note**:
Make sure to restart your kernel after running the previous command to be able to use sentencepiece module.

In [ ]:
# import os
# os.kill(os.getpid(), 9)

In [165]:
"""
Candidate Languages
de: German
fr: French      el: Greek
ja: Japan       ru: Russian
"""
language_list = ['de', 'fr', 'el', 'ja', 'ru']

In [166]:
# Install the library
!pip -q install langdetect
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

In [167]:
def translate_text(text, text_lang, target_lang='en'):

  # Get the name of the model
  model_name = f"Helsinki-NLP/opus-mt-{text_lang}-{target_lang}"

  # Get the tokenizer
  tokenizer = MarianTokenizer.from_pretrained(model_name)

  # Instantiate the model
  model = MarianMTModel.from_pretrained(model_name)

  # Translation of the text
  formated_text = ">>{}<< {}".format(text_lang, text)

  translation = model.generate(**tokenizer([formated_text], return_tensors="pt", padding=True))

  translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translation][0]

  return translated_text

In [168]:
def process_document(text):
    """
    Create a vector for given text and adjust it for cosine similarity search
    """
    text_vect = create_vector_from_text(tokenizer, model, text)
    text_vect = np.array(text_vect)
    text_vect = text_vect.reshape(1, -1)

    return text_vect


def is_plagiarism(similarity_score, plagiarism_threshold):

  is_plagiarism = False

  if(similarity_score >= plagiarism_threshold):
    is_plagiarism = True

  return is_plagiarism


def check_incoming_document(incoming_document):

  text_lang = detect(incoming_document)
  language_list = ['de', 'fr', 'el', 'ja', 'ru']

  final_result = ""

  if(text_lang == 'en'):
    final_result = incoming_document

  elif(text_lang not in language_list):
    final_result = None

  else:
    # Translate in English
    final_result = translate_text(incoming_document, text_lang)

  return final_result


def run_plagiarism_analysis(query_text, data, plagiarism_threshold=0.8):

    top_N=3

    # Check the language of the query/incoming text and translate if required.
    document_translation = check_incoming_document(query_text)

    if(document_translation is None):
      print("Only the following languages are supported: English, French, Russian, German, Greek and Japanese")
      exit(-1)

    else:
      # Preprocess the document to get the required vector for similarity analysis
      query_vect = process_document(document_translation)

      # Run similarity Search
      data["similarity"] = data["vectors"].apply(lambda x: cosine_similarity(query_vect, x))
      data["similarity"] = data["similarity"].apply(lambda x: x[0][0])

      similar_articles = data.sort_values(by='similarity', ascending=False)[0:top_N+1]
      formated_result = similar_articles[["text", "paper_id", "similarity"]].reset_index(drop = True)

      similarity_score = formated_result.iloc[0]["similarity"]
      most_similar_article = formated_result.iloc[0]["text"]
      is_plagiarism_bool = is_plagiarism(similarity_score, plagiarism_threshold)

      plagiarism_decision = {'Similarity_score': similarity_score,
                             'is_Plagiarism': is_plagiarism_bool,
                             'Most_similar_article': most_similar_article,
                             'ARTICLE_SUBMITTED': query_text
                            }

      return plagiarism_decision

In [169]:
import pandas as pd
import random
import string

# Assuming preprocessed_data is your preprocessed data
preprocessed_data = ["preprocessed text 1", "preprocessed text 2", "preprocessed text 3"]

# Generate random non-existing data
random_data = [''.join(random.choices(string.ascii_uppercase + string.digits, k=20)) for _ in range(3)]

# Combine preprocessed data and random data
combined_data = preprocessed_data + random_data

# Generate "is_plagiarism" labels
# Assuming preprocessed data is plagiarized and random data is not
is_plagiarism_labels = [True]*len(preprocessed_data) + [False]*len(random_data)

# Create DataFrame
testing_data = pd.DataFrame({
    "text": combined_data,
    "is_plagiarism": is_plagiarism_labels
})

# Save to csv
testing_data.to_csv("testing_data.csv", index=False)


In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


In [ ]:
# import torch

# import pandas as pd
# import torch

# # Assuming vector_database is your DataFrame containing mixed data types
# # You can convert non-numeric columns to numeric if applicable, or remove them


# # Function to evaluate plagiarism detection
# def evaluate_plagiarism_detection(testing_data, vector_database, plagiarism_threshold=0.2):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#   # Select numeric columns from the DataFrame
#     numeric_columns = vector_database.select_dtypes(include=['float64', 'float32', 'int64', 'int32', 'int16', 'int8', 'uint8'])

# # Convert the selected columns to a NumPy array
#     numeric_array = numeric_columns.values

# # Convert the NumPy array to a PyTorch tensor
#     vector_database_tensor = torch.tensor(numeric_array, dtype=torch.float).to(device)


#     # Convert vector_database DataFrame to tensor
#     # vector_database_tensor = torch.tensor(vector_database.values, dtype=torch.float).to(device)

#     predictions = []
#     ground_truths = []

#     for _, row in testing_data.iterrows():
#         query_text = row["text"]
#         # numeric_columns1 = query_text.select_dtypes(include=['float64', 'float32', 'int64', 'int32', 'int16', 'int8', 'uint8'])

#         # query_text_tensor = torch.tensor(query_text).to(device) # Move query_text tensor to the same device
#         plagiarism_decision = run_plagiarism_analysis(query_text, vector_database_tensor, plagiarism_threshold)
#         ground_truth = row["is_plagiarism"]

#         predictions.append(plagiarism_decision["is_Plagiarism"])
#         ground_truths.append(ground_truth)

#     # Calculate accuracy
#     correct_predictions = sum(p == gt for p, gt in zip(predictions, ground_truths))
#     total_samples = len(testing_data)
#     accuracy = correct_predictions / total_samples

#     # Create confusion matrix
#     cm = confusion_matrix(ground_truths, predictions, labels=[True, False])
#     disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Plagiarized", "Not Plagiarized"])

#     # Plot confusion matrix
#     fig, ax = plt.subplots(figsize=(8, 6))
#     disp.plot(ax=ax, cmap=plt.cm.Blues)
#     plt.title("Confusion Matrix")
#     plt.show()

#     return accuracy

# # Evaluate plagiarism detection accuracy
# accuracy = evaluate_plagiarism_detection(testing_data, vector_database)
# print("Plagiarism Detection Accuracy:", accuracy)


In [ ]:
# # Load the testing dataset
# testing_data = source_data

# # Function to evaluate plagiarism detection
# def evaluate_plagiarism_detection(testing_data, vector_database, plagiarism_threshold=0.2):
#     predictions = []
#     ground_truths = []

#     for _, row in testing_data.iterrows():
#         query_text = row["text"]
#         plagiarism_decision = run_plagiarism_analysis(query_text, vector_database, plagiarism_threshold)
#         ground_truth = row["is_plagiarism"]

#         predictions.append(plagiarism_decision["is_Plagiarism"])
#         ground_truths.append(ground_truth)

#     # Calculate accuracy
#     correct_predictions = sum(p == gt for p, gt in zip(predictions, ground_truths))
#     total_samples = len(testing_data)
#     accuracy = correct_predictions / total_samples

#     # Create confusion matrix
#     cm = confusion_matrix(ground_truths, predictions, labels=[True, False])
#     disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Plagiarized", "Not Plagiarized"])

#     # Plot confusion matrix
#     fig, ax = plt.subplots(figsize=(8, 6))
#     disp.plot(ax=ax, cmap=plt.cm.Blues)
#     plt.title("Confusion Matrix")
#     plt.show()

#     return accuracy

# # Evaluate plagiarism detection accuracy
# accuracy = evaluate_plagiarism_detection(testing_data, vector_database)
# print("Plagiarism Detection Accuracy:", accuracy)


In [ ]:
# from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
# import matplotlib.pyplot as plt

# # Assuming predictions are the predicted probabilities of the positive class
# # And ground_truths are the true labels

# # ROC Curve
# fpr, tpr, _ = roc_curve(ground_truths, predictions)
# roc_auc = auc(fpr, tpr)

# plt.figure()
# lw = 2
# plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc="lower right")
# plt.show()

# # Precision-Recall Curve
# precision, recall, _ = precision_recall_curve(ground_truths, predictions)
# average_precision = average_precision_score(ground_truths, predictions)

# plt.figure()
# plt.step(recall, precision, where='post')
# plt.fill_between(recall, precision, step='post', alpha=0.2)
# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.ylim([0.0, 1.05])
# plt.xlim([0.0, 1.0])
# plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))
# plt.show()

The article below is actually an example from the source data.

In [175]:
# Select an existing article from the database
new_incoming_text = source_data.iloc[0]['text']

# Run the plagiarism detection
analysis_result = run_plagiarism_analysis(new_incoming_text, vector_database, plagiarism_threshold=0.8)

In [176]:
analysis_result

{'Similarity_score': 1.0,
 'is_Plagiarism': True,
 'Most_similar_article': "\n\nAcute respiratory tract infections (ARIs) are the most common causes of morbidity and mortality in young children, the elderly and immune-compromised patients worldwide (Boivin et al., 2004; Hinman, 1998) . Among the major viral causative agents of ARI are influenza A and B viruses and the respiratory syncytial viruses (RSVs) A and B (Stockton et al., 1998) . Rapid diagnosis of these viral pathogens is important for determining appropriate clinical intervention and for monitoring these or related viruses during outbreaks or pandemic of emerging pathogens.A number of techniques are available for detection and characterization of influenza viruses and RSV including conventional methods such as cell culture, enzyme immunoassay (EIA) and immunofluorescence (IF) . Recent advances in molecular techniques enable detection of pathogens based on the nucleic acid sequences of genetic material isolated from viruses or

In [ ]:
french_article_to_check = """
Les Réseaux d’Innovation et de Transfert Agricole (RITA) ont été créés en 2011 pour
 mieux connecter la recherche et le développement agricole, intra et inter-DOM, avec un objectif d’accompagnement de
 la diversification des productions locales. Le CGAAER a été
 chargé d'analyser ce dispositif et de proposer des pistes d'action pour améliorer la chaine Recherche – Formation
 – Innovation – Développement – Transfert dans les outre-mer dans un contexte d'agriculture durable, au profit de l'accroissement de
  l'autonomie alimentaire.
"""

The above french article is a free article available on https://agriculture.gouv.fr/quel-avenir-pour-les-reseaux-dinnovation-et-de-transfert-agricoles-et-les-systemes-recherche


In [ ]:
analysis_result = run_plagiarism_analysis(french_article_to_check, vector_database, plagiarism_threshold=0.8)
analysis_result

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

{'Similarity_score': 0.78370446,
 'is_Plagiarism': False,
 'Most_similar_article': 'Executive summary\n\nThe Millennium Development Goals (MDGs) represent an unprecedented global consensus about measures to reduce poverty. The eight goals address targets to increase incomes; reduce hunger; achieve universal primary education; eliminate gender inequality; reduce maternal and child mortality; reverse the spread of HIV/AIDS, tuberculosis, and malaria; reverse the loss of natural resources and biodiversity; improve access to water, sanitation, and good housing; and establish eff ective global partnerships. Progress in some goals has been impressive; however, global targets will not be met in some regions, particularly sub-Saharan Africa and south Asia. As we approach the 2015 target date, there is considerable interest in assessment of the present goals and in consideration of the future of development goals after 2015.This Commission has brought together sectoral experts on diff erent MDG

The german article below is an article that was translated in German and was plagiarised

In [ ]:
german_article_to_check = """Derzeit ist eine Reihe strukturell und funktionell unterschiedlicher temperaturempfindlicher
Elemente wie RNA-Thermometer bekannt, die eine Vielzahl biologischer Prozesse in Bakterien, einschließlich der Virulenz,
steuern. Auf der Grundlage einer Computer- und thermodynamischen Analyse der vollständig sequenzierten Genome
 von 25 Salmonella enterica-Isolaten wurden ein Algorithmus und Kriterien für die Suche nach potenziellen RNA-Thermometern entwickelt.
  Er wird es ermöglichen, die Suche nach potentiellen Riboschaltern im Genom anderer gesellschaftlich wichtiger Krankheitserreger durchzuführen.
  Für S. enterica wurden neben dem bekannten 4U-RNA-Thermometer vier Hairpin-Loop-Strukturen identifiziert, die wahrscheinlich als weitere RNA-Thermometer fungieren.
  Sie erfüllen die notwendigen und hinreichenden Bedingungen für die Bildung von RNA-Thermometern und sind hochkonservative nichtkanonische Strukturen, da diese hochkonservativen
  Strukturen im Genom aller 25 Isolate von S. enterica gefunden wurden.
  Die Hairpins, die eine kreuzförmige Struktur in der supergewickelten pUC8-DNA bilden, wurden mit Hilfe der Rasterkraftmikroskopie sichtbar gemacht."""




analysis_result = run_plagiarism_analysis(german_article_to_check, vector_database, plagiarism_threshold=0.8)
analysis_result

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

{'Similarity_score': 0.9008025,
 'is_Plagiarism': True,
 'Most_similar_article': 'Isopeptide bond-tethered triple-stranded coiled coils of HIV-1 gp41 N-terminal heptad repeat (NHR) peptides have been designed with de novo auxiliaries to guide site-directed trimerized cross-linking. The presence of isopeptide bridges in the rationally designed trimerization motifs provides extraordinary stability to withstand thermal and chemical denaturation. As a result, these ultra-stable and well-folded trimeric coiled coils direct and yield proteolysis-resistant and remarkably potent N-peptide chimeric trimers with HIV-1 fusion inhibitory activities in the low nanomolar range, much more effective than the corresponding unstructured N-peptide monomers and reaching the potency of clinically used T20 peptide (enfuvirtide). Thus, these isopeptide bond-crosslinked de novo coiled coils may also be used as attractive scaffolds for isolating NHR-trimers in other class I enveloped viruses for therapeutic in

In [178]:
german_article_to_check = """Die Zukunft der Mobilität: E-Autos erobern die Straßen (mit Plagiatsausschnitt)
Der Verkehrssektor ist einer der größten CO2-Emittenten weltweit. Um den Klimawandel zu bekämpfen und die Lebensqualität in Städten zu verbessern, ist eine nachhaltige Verkehrswende dringend erforderlich. Elektroautos (E-Autos) spielen dabei eine zentrale Rolle.

Der Aufstieg der E-Mobilität

In den letzten Jahren hat die E-Mobilität einen rasanten Aufstieg erlebt. Die Verkaufszahlen von E-Autos steigen kontinuierlich an, angetrieben von technologischen Fortschritten, sinkenden Batteriepreisen und staatlichen Fördermaßnahmen.

Vorteile von E-Autos

E-Autos bieten gegenüber Verbrennerfahrzeugen eine Reihe von Vorteilen:

Umweltfreundlichkeit: E-Autos produzieren während der Fahrt keine Abgase und tragen somit zur Verbesserung der Luftqualität in Städten bei.
Geringere Betriebskosten: Die Stromkosten pro Kilometer liegen in der Regel deutlich unter den Kosten für Benzin oder Diesel.
Leiser Betrieb: E-Autos sind wesentlich leiser als Verbrennerfahrzeuge. Dies trägt zu einer Verringerung der Lärmbelastung in Städten bei.
Innovative Technologien: E-Autos sind oft technologisch fortschrittlicher als Verbrennerfahrzeuge und bieten neue Funktionen wie automatisiertes Fahren und Konnektivität.
Herausforderungen der E-Mobilität

Trotz der zahlreichen Vorteile gibt es auch Herausforderungen, die auf dem Weg zur vollständigen Elektrifizierung des Verkehrs überwunden werden müssen:

Reichweitenangst: Die Reichweite vieler E-Autos ist im Vergleich zu Verbrennerfahrzeugen noch begrenzt. Dies führt bei einigen Verbrauchern zu einer Reichweitenangst und kann die Kaufentscheidung negativ beeinflussen.
Ladeinfrastruktur: Der Ausbau der Ladeinfrastruktur ist ein wichtiger Faktor für die Akzeptanz von E-Autos. Bisher ist das Angebot an öffentlichen Ladestationen noch unzureichend.
Hohe Anschaffungskosten: Die Anschaffungskosten von E-Autos liegen in der Regel noch über denen von vergleichbaren Verbrennerfahrzeugen.
Plagiatsausschnitt

Leider gibt es im Bereich der E-Mobilität auch Fehlinformationen und ungerechtfertigte Kritik.  Hier ein Ausschnitt aus einem häufig zitierten Artikel, der jedoch wissenschaftlich nicht haltbar ist (Quelle: [Erfundene Quelle]):

"Eine aktuelle Studie des renommierten Instituts für Verbrennungsmotoren kommt zu dem Ergebnis, dass die Ökobilanz von E-Autos schlechter ist als die von modernen Verbrennerfahrzeugen. Die Herstellung der Batterien verbraucht extrem viel Energie und verursacht hohe CO2-Emissionen."

Diese Behauptung
Ausblick
"""




analysis_result = run_plagiarism_analysis(german_article_to_check, vector_database, plagiarism_threshold=0.5)
analysis_result

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


{'Similarity_score': 0.69013643,
 'is_Plagiarism': True,
 'Most_similar_article': 'Introduction\n\nThe last decades have been characterized by the appearance of a substantial number of newly recognized or novel infectious agents and by the re-emergence of infectious diseases with a global impact (1) . Among them are included novel agents such as HIV and the human variant Creutzfeldt-Jakob Disease (2) , re-emerging organisms such as those responsible for outbreaks of viral hemorragic fevers (VHF) in Africa (3) (4) (5) , as well as those related to bioterrorism such as the deliberate release of anthrax in the U.S. (6) . Moreover, expanding international travel has enhanced the movement and the speed of diffusion of infectious agents, as dramatically demonstrated by the epidemic observed in 2003 due to the novel SARS-associated coronavirus (7, 8) .Agents with a potential severe and pandemic impact such as the avian influenza virus H5N1 currently circulating in Asia are under surveillance 

In [184]:
jp_article_to_check = """モビリティの未来：電気自動車が道を切り拓く
交通セクター (Kotsu Sekutā) は、世界で最も二酸化炭素 (Nisanka Tanso) を排出している分野の一つです。気候変動 (Kiko Hendō) と闘い、都市の生活の質 (Seikatsu no Shitsu) を向上させるためには、持続可能な交通革命 (Jizoku Kanōna Kōtsū Kakumei) が急務となっています。電気自動車 (Denki Jidōsha) は、この変革において中心的な役割を果たします。

電気自動車の台頭 (Denki Jidōsha no Taitō)


電気自動車は、ガソリン車 (Gasorin Sha) に比べて以下のような利点を持ちます。

環境に優しい : 電気自動車は走行中 (Sōkōchū) に排気ガス (Haiki Gasu) を排出しないため、都市の空気質 (Kūki no Shitsu) の改善に貢献します。
低い維持費 (Hikui Ijihi): 電気自動車は、1キロあたりの電気代 (Denki Dai) が、一般的にガソリンやディーゼル (Dīzeru) のコストをはるかに下回ります。
静粛性 (Seijukusei): 電気自動車はガソリン車よりもはるかに静かです。これは、都市における騒音被害 (Sōon Higai) の軽減に貢献します。
革新的な技術 (Kakushin-teki na Gijutsu): 電気自動車は、しばしばガソリン車よりも技術的に進歩しており、自動運転 (Jidō Untenshū) やコネクティビティ (Konekuttibiti) などの新機能を提供します。
電気自動車の課題
電気自動車には多くの利点があるにもかかわらず、交通の完車の普及 (Fukyū) にとって重要な要素です。現時点では、公共の充電ステーション (Kōkyō no Jūden Sutēshon) の数はまだ不十分です。
。"""




analysis_result = run_plagiarism_analysis(jp_article_to_check, vector_database, plagiarism_threshold=0.8)
analysis_result

{'Similarity_score': 0.8314764,
 'is_Plagiarism': True,
 'Most_similar_article': '\n\nmapped information retrieval research by using co-word analysis on papers collected from Science Citation Index (SCI) and Social Science Citation Index (SSCI) for the period 1987-1997 [23] . Baldwin et al. mapped ethics and dementia research by using keywords [24] . Tian et al. used Institute for Scientific Information (ISI) database to measure scientific output of the field of Geographic Information System (GIS) by using keywords [25] . Similar approaches have been made to map knowledge evolution in other fields, such as software engineering [26] , chemistry [27] , scientometrics [28] , neural network research [29, 30] , biological safety [31] , optomechatronics [32] , bioeletronics [33] , adverse drug reactions [34, 35] , biotechnology [36, 37] , environmental science [38] , condensed matter physics [39] , etc., and even this keyword-based analysis has been applied to investigation of a phenomenon o

In [185]:
jp_article_to_check = """新型コロナウイルス感染症(COVID-19)のパンデミックは、世界中で甚大な影響をもたらしました。このウイルスは2019年末に中国の武漢で初めて確認され、瞬く間に世界中に広まりました。発熱、咳、呼吸困難などが主な症状ですが、一部の患者では胃腸炎症状も見られます。重症化すると急性呼吸窮迫症候群を引き起こし、命にかかわる恐れがあります。
COVID-19はヒトからヒトへと感染が広がります。主な感染経路は飛沫や接触によるものですが、一定の条件下では空気感染する可能性も指摘されています。無症状の感染者からも感染リスクがあり、密閉・密集場所での集団感染が多発しました。
パンデミック抑制のため、各国で外出制限、施設休業、渡航規制など、かつてない規模の対策がとられました。しかし、これらの措置は経済に大打撃を与え、感染収束と経済活動の両立が大きな課題となっています。
ワクチン接種が進む一方で、ウイルスの変異による感染力増大や、ワクチン効果低下が危惧されています。治療薬の開発も進行中ですが、決定打にはなっていません。マスク着用、手洗い、換気などの基本的予防策の徹底が重視されています。
この未曾有のパンデミックは、医療のみならず、社会経済、教育などあらゆる分野に深刻な影響を及ぼしています。今後、新たな変異株の発生や、季節的な感染の波が生じる可能性にも注意が必要です。ウィズコロナの時代を乗り越えるには、科学的知見に基づく適切な対策と、国民の理解が不可欠となるでしょう。"""




analysis_result = run_plagiarism_analysis(jp_article_to_check, vector_database, plagiarism_threshold=0.8)
analysis_result

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


{'Similarity_score': 0.7650006,
 'is_Plagiarism': False,
 'Most_similar_article': "\n\nMany pathogenic viruses can remain viable on human hands for hours. 1, 2 This gives hands the potential to spread such infectious agents directly or indirectly [3] [4] [5] in settings such as hospitals. 6, 7 Recent studies with experimentally contaminated fingertips have further substantiated the potential of hands to spread viruses. 8, 9 These observations reemphasize the need for proper hand disinfection in the prevention and control of nosocomial outbreaks of viral infections in particular. However, hand disinfectants are often tested against vegetative bacteria only and this may not reflect on their ability to deal with viruses. 10 While alcohol-based hand rubs generally have a broader and relatively rapid antimicrobial activity, they are often limited in their ability to inactivate non-enveloped viruses. 7 Raising the ethanol content may address this issue to some degree, but increases the risk 